In [1]:
import os, re, csv
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi
from soyspacing.countbase import CountSpace

In [2]:
def csv2data(*df):
    
    """
    csv 파일은 필수적으로 내용(질문)을 가지고 있어야한다.
    
    """

    df_lst = list(df)
    merged_df_lst = []
    for df in df_lst:
        col = df.columns.tolist()

        if '내용' not in col:
            print("내용이 없습니다")
        else:
            pass

        col_nme = []
        not_need = []
        for name in col:
            if '내용' == name:
                col_nme.append('내용')
                col.remove(name)
            else:
                pass

        for name in col:
            if '댓글' == name:
                col_nme.append('댓글')
                col.remove(name)
            else:
                pass


        df = df.drop(col, axis=1)
        if '댓글' not in df.columns.tolist():
            df['댓글'] = np.nan
        df['TAGSET'] = np.nan
        df.rename(columns={"내용":"QUESTION", "댓글":"ANSWER"}, inplace=True)
        
        merged_df_lst.append(df)

    # 데이터 병합
    merged_df = pd.concat(merged_df_lst)
    merged_df.reset_index(drop=True, inplace=True)
    merged_df = merged_df[['QUESTION', "ANSWER", "TAGSET"]]

    return merged_df

In [3]:
def preprocess(merged_df, space_model_path, stop_word_path, result_file_name):
    """
    import os, re, csv
    import pandas as pd
    import numpy as np
    from kiwipiepy import Kiwi
    from soyspacing.countbase import CountSpace

    pip install soyspace
    pip install kiwipiepy

    model : soy_space_model (자료 폴더에있음)
 model : soy_space_model (자료 폴더에있음)
    stop_word : 자료 파일에 있음

    """
        
    model = CountSpace()
    model.load_model(space_model_path, json_format=False)

    # 질문 리스트화
    question_list = merged_df['QUESTION'].tolist()

    # 띄어쓰기 시작
    soy_corrected_question = []
    for x in question_list:
        corrected = model.correct(x)
        soy_corrected_question.append(corrected)

    spaced_quest = []
    for quest, tag in soy_corrected_question:
        if type(quest) == str:
            spaced_quest.append(quest)
        else:
            continue


    # 정규표현식으로 정규화
    norm_content = []
    for text in spaced_quest:
        punct = re.compile(r'[\~\'\"\.\,\?\!\:\;\)\(\+]+') 
        norm_text = re.sub(punct, "", text)
        norm_text = re.sub(r'(안녕)+(하세요)?', '', norm_text)
        norm_text = re.sub(r'(조교)[님]?', "", norm_text)
        norm_text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', norm_text) # ㅠ하나 쓰는 놈들꺼 제거...
        norm_text = re.sub( r"\s{2,}"," ", norm_text) 
        norm_text = re.sub( r"\A\s+",'', norm_text)
        norm_text = re.sub(r"([가-힣]+)[여용염]",r"\1요",norm_text) # 용용체 제거 슈바끄...
        norm_text = re.sub(r"\b이중[^전공]*?\b","이중전공",norm_text) # 이중으로만 적혀있는거 변형
        norm_text = re.sub(r"\b부전[^공]*?\b","부전공",norm_text) # 부전으로만 적혀있는거 변형

        # 줄인말제거
        norm_text = re.sub(r"\b전심[가-힣]*\b", "전공심화", norm_text)
        norm_text = re.sub(r"\b융전[가-힣]*\b", "융합전공", norm_text)
        norm_text = re.sub(r'\b연계전공\b', '융합전공', norm_text) # 단어 통일화
        
        norm_content.append(norm_text)

    # 데이터프레임에 병합
    merged_df['QUESTION'] = norm_content

    # 형태소 분석

    kiwi = Kiwi()

    #대분류 사전 추가
    kiwi.add_user_word('이중전공', 'DM', 20.0)
    kiwi.add_user_word('융합전공', 'UM', 20.0)
    kiwi.add_user_word('전공심화', 'MD', 20.0)
    kiwi.add_user_word('부전공', 'SM', 20.0)
    kiwi.add_user_word('연계전공', 'UM', 20.0)

    kiwi.prepare()

    ## 형태소 분석
    sent = ''
    lem_cont = []
    for quest in norm_content:
        ## 형태소만 떼어내기
        temp_tagging = [x[0] for x in kiwi.analyze(quest, top_n=1)]
        inner_temp = ["{}/{}".format(word, tag) for word, tag, score1, score2 in temp_tagging[0]]
        lem_cont.append(inner_temp)

    ## 불용어 처리 ##

    stop_file = open(stop_word_path,'r', encoding = 'utf8')
    lines = stop_file.readlines()
    stop_file.close()
    stop_words = lines[0].split()

    listed_cont = [[comb.split('/') for comb in sent] for sent in lem_cont]

    main_texts = []
    non_mean = []
    for sent in listed_cont:
        texts = []
        for word in sent:
            # 한 글자 제거
            if len(word[0]) > 1:
                # 조사 등 불필요한 거 제거
                if 'N' in word[-1]:
                    texts.append(word[0])
                elif '@' in word[-1]:
                    texts.append(word[0])
                elif 'V' in word[-1]:
                    texts.append(word[0])
                else:
                    non_mean.append(word[0])
            else:
                non_mean.append(word[0])
        
        main_texts.append(texts)

    lemmatized_question = []

    for noun in main_texts:
        txt = ' '.join(noun)
        lemmatized_question.append(txt)

    merged_df['LEMMA'] = lemmatized_question

    ##Tagset 달기
    tagset = []
    for sent in main_texts:
        tags = []
        for word in sent:
            if '이중전공' == word:
                tags.append("DM")
            elif '부전공' == word:
                tags.append("SM")
            elif '전공심화' == word:
                tags.append("MD")
            elif '융합전공' == word:
                tags.append("UM")
            else:
                tags.append("UK")
        tagset.append(tags)

    tagsets = []
    for sent_tag in tagset:
        tagsets.append(list(set(sent_tag)))

    for tags in tagsets:
        if len(tags) > 1:
            del tags[-1]

    merged_df['TAGSET'] = tagsets

    # csv로 출력
    path = os.getcwd()
    merged_df.to_csv(path + "\\" + result_file_name + ".csv", index=False, encoding="utf-8")

    return merged_df

In [6]:
# eta_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\질문.csv", encoding='ANSI')
# eta2_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\질문2.csv", encoding='ANSI')
# dm_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\이중전과게시판_이중검색.csv", encoding='UTF-8')
double_major_df = pd.read_excel(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\이중전공_howls.xlsx")
um_df = pd.read_excel(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\융합전공_질문_답변.xlsx")

merged_df = csv2data(double_major_df, um_df)

In [8]:
space = r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\models\soy_big_space_model"
stop = r'E:\Programming\python\NLP\howls_nlp\Chatbot\자료\stop_words.txt'

preprocess(merged_df, space, stop, "preprocessed_made_data")

,QUESTION,ANSWER,TAGSET,LEMMA
0,서울캠에서 네덜란드어전공을 들을 수 있나요,아니요,[UK],서울캠 네덜란드어 전공
1,서울캠에서 경영학 들을 수 있나요,아니요,[UK],서울캠 경영학
2,언어와공학 이중전공 서울캠 수업 들을 수 있어요,네. 들을 수 있습니다.,[DM],언어 공학 이중전공 서울 수업
3,편입생도 이중전공 할 수 있어요,"네. 필수는 아니지만 재학 연한 내에서 이중전공, 부전공 또는 전공 심화 과정을 이...",[DM],편입 생도 이중전공
4,네덜란드어전공은 어느캠퍼스에서 들어,서울캠퍼스에서 이수해야 합니다.,[UK],네덜란드어 전공 캠퍼스
...,...,...,...,...
120,광역특화전공이 뭔가요,국가주도 CK-Ⅱ 사업의 일환으로 2015학년도부터 신설되는 ‘광역특화’ 융합전 공...,[UK],광역 특화 전공
121,디지털인문학전공이 뭔가요,한국의 인문학적 전통과 문화에 대한 심층적 지식을 바탕으로 디지털 매체를 활용한 해...,[UK],디지털 인문학 전공
122,디지털인문학전공의 연계기초과목엔 뭐가 있나요,"한국철학사(철학과), 한국사입문(사학과) 또는 한국문화사(사학 부전공), KFL문법...",[UK],디지털 문학 전공 연계 기초 과목
123,디지털인문학전공 연계기초 과목 학점인정은 어떻게 하나요,수강신청 후 학사종합지원센터 단과대 교학파트(본인 1전공 소속 기준)에 방문하여 이...,[UK],디지털 인문학 전공 연계 기초 과목 학점 인정 어떻


In [12]:
origin_df = pd.read_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\results\preprocessed_data.csv", encoding="cp949")
new_merged_df = pd.concat([origin_df, merged_df])
new_merged_df.reset_index(drop=True, inplace=True)
new_merged_df.to_csv(r"E:\Programming\python\NLP\howls_nlp\Chatbot\자료\results\Q&A.csv", index=False, encoding="utf-8")